In [3]:
#!pip install datasets
#!pip install huggingface_hub

   ---------------------------------------- 0.0/510.5 kB ? eta -:--:--
   ---- ----------------------------------- 61.4/510.5 kB 3.2 MB/s eta 0:00:01
   ----------------- ---------------------- 225.3/510.5 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------  501.8/510.5 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 510.5/510.5 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/365.3 kB ? eta -:--:--
   --------------------------------------- 365.3/365.3 kB 22.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
    --------------------------------------- 0.6/24.8 MB 11.8 MB/s eta 0:00:03
   -- ------------------------------------- 1.6/24.8 MB 25.1 MB/s eta 0:00:01
   ----- ---------------------------------- 3.6/24.8 MB 28.8 MB/s eta 0:00:01
   -

In [1]:
import pandas as pd
from datasets import load_dataset
import random
from evaluate import load
import shutil
import os
from data_engineering import *

### Downloading whole bigos v2 polish ASR dataset

In [22]:
data = load_dataset("amu-cai/pl-asr-bigos-v2",'all', 'all')


c:\Users\Eryk\anaconda3\envs\Magisterka\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for amu-cai/pl-asr-bigos-v2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/amu-cai/pl-asr-bigos-v2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/55 [00:00<?, ?it/s]

In [31]:
type(data['train'])

datasets.arrow_dataset.Dataset

#### Creating dataframe from the training set with 5000 randomly chosen examples


In [24]:
df_model_testing = creating_random_split_df(data['train'], 500)
df_model_testing

ValueError: Input must be a DataFrame or a dictionary.

#### Moving audio files to the folder in project

In [29]:
target_folder_path = './data/testing_batch/clear/'
copy_files(df_model_testing, 'audiopath_local', target_folder_path)
df_model_testing['audiopath_project'] = target_folder_path + df_model_testing['audiopath_bigos']

File mozilla-common_voice_15-23-train-2856-01818.wav copied successfully.
File pjatk-clarin_studio-15-train-0457-00001.wav copied successfully.
File pjatk-clarin_mobile-15-train-0083-00007.wav copied successfully.
File pwr-maleset-unk-train-0001-03097.wav copied successfully.
File mozilla-common_voice_15-23-train-2862-00017.wav copied successfully.
File google-fleurs-22-train-0001-00551.wav copied successfully.
File mailabs-corpus_librivox-19-train-2007-00141.wav copied successfully.
File mozilla-common_voice_15-23-train-2856-01676.wav copied successfully.
File fair-mls-20-train-0024-03775.wav copied successfully.
File fair-mls-20-train-0009-02010.wav copied successfully.
File mailabs-corpus_librivox-19-train-1012-00288.wav copied successfully.
File mozilla-common_voice_15-23-train-2915-00035.wav copied successfully.
File fair-mls-20-train-0024-00192.wav copied successfully.
File fair-mls-20-train-0021-00001.wav copied successfully.
File fair-mls-20-train-0024-01130.wav copied successf

### Saving dataframe to parquet

In [37]:
df_testing = df_model_testing[['audioname','dataset','ref_orig','sampling_rate','audiopath_local','audiopath_project']]
df_testing.to_parquet('testing_batch_df.parquet.gzip', compression = 'gzip')

In [54]:
df_urban_sounds = pd.read_csv('./data/UrbanSound8K/metadata/UrbanSound8K.csv')
df_urban_shuffled = df_urban_shuffled[~df_urban_shuffled['classID'].isin([2, 9, 5])]
df_urban_shuffled = creating_random_split_df(df_urban_sounds,3500)

In [56]:
# Function to create the file path
def create_file_path(row, folder_path):
    folder_number = row['fold']
    file_name = row['slice_file_name']
    file_path = os.path.join(folder_path, f'fold{folder_number}', file_name)
    return file_path

base_path = '.\\data\\UrbanSound8K\\audio\\'

# Apply the function to create the new column
df_urban_shuffled['audio_path'] = df_urban_shuffled.apply(create_file_path, axis=1, folder_path=base_path)


In [62]:
visc_folder_path = '.\\data\\VISC Dataset SON\\'


file_paths = []
class_ids = []


# Traverse the directory
for filename in os.listdir(visc_folder_path):
    # Join the folder path with the filename to get the full file path
    file_path = os.path.join(visc_folder_path, filename)
    
    # Extract the class ID from the file name
    class_id = int(filename.split()[0])
    
    # Append the values to the lists
    file_paths.append(file_path)
    class_ids.append(class_id)

# Create a DataFrame
df = pd.DataFrame({'file_path': file_paths, 'class_id': class_ids})



In [67]:
visc_noises_dataframe = creating_random_split_df(df,1500)

visc_noises_dict = {1 : 'bus_interior',
                    2 : 'minibus_interior',
                    3 : 'pickup_interior',
                    4 : 'sports_car_interior',
                    5 : 'jeep_interior',
                    6 : 'truck_interior',
                    7 : 'crossover_interior',
                    8 : 'other_car_interior'}
visc_noises_dataframe['class'] = visc_noises_dataframe['class_id'].map(visc_noises_dict)

In [68]:
visc_noises_dataframe

,file_path,class_id,class
0,.\data\VISC Dataset SON\3 (503).wav,3,pickup_interior
1,.\data\VISC Dataset SON\2 (324).wav,2,minibus_interior
2,.\data\VISC Dataset SON\1 (544).wav,1,bus_interior
3,.\data\VISC Dataset SON\6 (829).wav,6,truck_interior
4,.\data\VISC Dataset SON\5 (576).wav,5,jeep_interior
...,...,...,...
1495,.\data\VISC Dataset SON\3 (432).wav,3,pickup_interior
1496,.\data\VISC Dataset SON\4 (69).wav,4,sports_car_interior
1497,.\data\VISC Dataset SON\7 (69).wav,7,crossover_interior
1498,.\data\VISC Dataset SON\7 (152).wav,7,crossover_interior


In [46]:
# Function to create the file path
def create_file_path(row):
    folder_number = row['fold']
    file_name = row['slice_file_name']
    file_path = os.path.join('.\\data\\UrbanSound8K\\audio\\', f'fold{folder_number}', file_name)
    return file_path

# Apply the function to create the new column
df_urban_shuffled['audio_path'] = df_urban_shuffled.apply(create_file_path, axis=1)



In [50]:
df_urban_shuffled = df_urban_shuffled[~df_urban_shuffled['classID'].isin([2, 9, 5])]

In [51]:
df_urban_shuffled['classID'].unique()

array([6, 0, 7, 1, 4, 3, 8], dtype=int64)

In [81]:
df_urban_shuffled.dtypes

slice_file_name     object
fsID                 int64
start              float64
end                float64
salience             int64
fold                 int64
classID              int64
class               object
audio_path          object
dtype: object

In [72]:
visc_noises_dataframe

,file_path,class_id,class
0,.\data\VISC Dataset SON\3 (503).wav,3,pickup_interior
1,.\data\VISC Dataset SON\2 (324).wav,2,minibus_interior
2,.\data\VISC Dataset SON\1 (544).wav,1,bus_interior
3,.\data\VISC Dataset SON\6 (829).wav,6,truck_interior
4,.\data\VISC Dataset SON\5 (576).wav,5,jeep_interior
...,...,...,...
1495,.\data\VISC Dataset SON\3 (432).wav,3,pickup_interior
1496,.\data\VISC Dataset SON\4 (69).wav,4,sports_car_interior
1497,.\data\VISC Dataset SON\7 (69).wav,7,crossover_interior
1498,.\data\VISC Dataset SON\7 (152).wav,7,crossover_interior


In [78]:
noise_combined_df = pd.merge(visc_noises_dataframe[['file_path', 'class']],df_urban_shuffled[['audio_path', 'class']], left_index=True, right_index=True, how='outer')

noise_combined_df.rename(columns={'file_path': 'noise_path'}, inplace=True)

In [79]:
noise_combined_df

,noise_path,class_x,audio_path,class_y
0,.\data\VISC Dataset SON\3 (503).wav,pickup_interior,.\data\UrbanSound8K\audio\fold2\156893-7-0-0.wav,jackhammer
1,.\data\VISC Dataset SON\2 (324).wav,minibus_interior,.\data\UrbanSound8K\audio\fold6\39884-5-0-6.wav,engine_idling
2,.\data\VISC Dataset SON\1 (544).wav,bus_interior,.\data\UrbanSound8K\audio\fold1\180937-7-3-17.wav,jackhammer
3,.\data\VISC Dataset SON\6 (829).wav,truck_interior,.\data\UrbanSound8K\audio\fold3\12647-3-3-0.wav,dog_bark
4,.\data\VISC Dataset SON\5 (576).wav,jeep_interior,.\data\UrbanSound8K\audio\fold1\203440-3-0-6.wav,dog_bark
...,...,...,...,...
3495,NaN,NaN,.\data\UrbanSound8K\audio\fold6\94632-5-0-2.wav,engine_idling
3496,NaN,NaN,.\data\UrbanSound8K\audio\fold3\62837-7-0-3.wav,jackhammer
3497,NaN,NaN,.\data\UrbanSound8K\audio\fold8\113203-5-0-0.wav,engine_idling
3498,NaN,NaN,.\data\UrbanSound8K\audio\fold6\182474-1-0-0.wav,car_horn


### Loading work dataframe from parquet

In [2]:
df_testing_models = pd.read_parquet('testing_batch_df.parquet.gzip') 
df_testing_models

,audioname,dataset,ref_orig,sampling_rate,audiopath_local,audiopath_project
0,mozilla-common_voice_15-23-train-2856-01818,mozilla-common_voice_15-23,"Jest także trzecia sprawa, która w czasie tej ...",16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...
1,pjatk-clarin_studio-15-train-0457-00001,pjatk-clarin_studio-15,dżuma wziernik przemianę księdzu krzywdzen...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_studio...
2,pjatk-clarin_mobile-15-train-0083-00007,pjatk-clarin_mobile-15,w piątek po południu była przesłuchiwana przez...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_mobile...
3,pwr-maleset-unk-train-0001-03097,pwr-maleset-unk,jeśli chcesz zostanę w domu,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pwr-maleset-unk-tra...
4,mozilla-common_voice_15-23-train-2862-00017,mozilla-common_voice_15-23,Tekst nie opiera się na żadnych podstawach nau...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...
...,...,...,...,...,...,...
4995,mailabs-corpus_librivox-19-train-2023-00011,mailabs-corpus_librivox-19,Nareszcie zniecierpliwiony kazał zamurować okn...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mailabs-corpus_libr...
4996,pjatk-clarin_studio-15-train-0289-00016,pjatk-clarin_studio-15,dostała za ten reportaż nagrodę pulicera ...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_studio...
4997,mozilla-common_voice_15-23-train-2846-00448,mozilla-common_voice_15-23,Dotyczy ona zasadniczo dwóch kwestii,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/mozilla-common_voic...
4998,pjatk-clarin_mobile-15-train-0035-00018,pjatk-clarin_mobile-15,każdy starał się odlecieć najbliższym samolote...,16000,C:\Users\Eryk\.cache\huggingface\datasets\down...,./data/testing_batch/clear/pjatk-clarin_mobile...


## Noises 

#### Shuffling noises from Urban Sounds and VISC Dataset to create one noises dataset

In [3]:
df_urban_sounds = pd.read_csv('./data/UrbanSound8K/metadata/UrbanSound8K.csv')
df_urban_sounds

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [ ]:
df_urban_shuffled.

In [15]:
df_urban_shuffled = creating_random_split_df(df_urban_sounds,5000)

In [82]:
df_urban_shuffled

,slice_file_name,fsID,start,end,salience,fold,classID,class,audio_path
0,156893-7-0-0.wav,156893,0.948287,2.203372,2,2,7,jackhammer,.\data\UrbanSound8K\audio\fold2\156893-7-0-0.wav
1,39884-5-0-6.wav,39884,11.085243,15.085243,1,6,5,engine_idling,.\data\UrbanSound8K\audio\fold6\39884-5-0-6.wav
2,180937-7-3-17.wav,180937,313.084561,317.084561,2,1,7,jackhammer,.\data\UrbanSound8K\audio\fold1\180937-7-3-17.wav
3,12647-3-3-0.wav,12647,6.682172,10.682172,1,3,3,dog_bark,.\data\UrbanSound8K\audio\fold3\12647-3-3-0.wav
4,203440-3-0-6.wav,203440,26.509202,30.509202,2,1,3,dog_bark,.\data\UrbanSound8K\audio\fold1\203440-3-0-6.wav
...,...,...,...,...,...,...,...,...,...
3495,94632-5-0-2.wav,94632,24.604670,28.604670,2,6,5,engine_idling,.\data\UrbanSound8K\audio\fold6\94632-5-0-2.wav
3496,62837-7-0-3.wav,62837,1.500000,5.500000,1,3,7,jackhammer,.\data\UrbanSound8K\audio\fold3\62837-7-0-3.wav
3497,113203-5-0-0.wav,113203,43.494706,47.494706,1,8,5,engine_idling,.\data\UrbanSound8K\audio\fold8\113203-5-0-0.wav
3498,182474-1-0-0.wav,182474,1.265010,2.605597,1,6,1,car_horn,.\data\UrbanSound8K\audio\fold6\182474-1-0-0.wav


### Model bark whisper v3 Large


In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [7]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
#whisper_results = []
#for i in range(len(df_whisper)):
#    sample = df_whisper['audiopath_local'][i]
#    result = pipe(sample)
#    whisper_results.append(result['text'])

#df_whisper['whisper_pred'] = whisper_results

for i in range(10):
    sample = df_whisper['audiopath_local'][i]
    result = pipe(sample)
    print(result['text'])


 Ogólnie można powiedzieć, że w ciągu ostatnich dwóch lat nasze stosunki się poprawiły.
 Ziarna mniejsze prawdopodobnie flokulują, a otrzymane wyniki określają raczej kinetykę floku galenowych.
 Niezależnie od tego, którą wersję oprogramowania Państwo nabyli, mogą Państwo sporządzać tylko jedną kopię archiwalną oprogramowania.
 Nic dziwnego oburzył przeciw sobie wszystkich fabrykantów jedwabnych i bawełnianych, a także kupców, którzy sprzedają im towary.
 Ja oraz pozostali konserwatyści brytyjscy zgadzamy się z większością przedmiotowego sprawozdania.
 Była to zatem sprawa między Niemcami a ligą, a nie między Gdańskiem a Polską. Tłem sporu była chęć manifestacji III Rzeszy przeciw mieszaniu się ligi wewnętrzno-politycznych sprawy Gdańsk.
 Ja zaś znowu dręczyłam się myślą, że stałem się przyczyną jej nieszczęścia.
 zapewne rzekł wokulski ponamyśle bo i na co trzysta tysięcy rubli tobie któremu wystarcza tysiąc na rok to prawda rzecki zbliżył swoje usta do jego ucha jeszcze ci powiem że 

In [31]:
len(whisper_results)


500

In [42]:
df_whisper['ref_orig']
references = []
references = df_whisper['ref_orig'][:500].to_list()

In [37]:
predictions = whisper_results

In [43]:
wer = load("wer")
wer_score = wer.compute(predictions=predictions, references=references)
print(wer_score)

0.1750317972801096


### Model jonatasgrosman/wav2vec2-large-xlsr-53-polish

In [11]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [15]:

LANG_ID = "pl"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-polish"
SAMPLES = 5


processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["audiopath_local"], sr=16_000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["ref_orig"].upper()
    return batch

df_test_wav = pd.DataFrame()
df_test_wav = speech_file_to_array_fn(df_whisper[0:5])
inputs = processor(df_test_wav['speech'], sampling_rate=16_000, return_tensors="pt", padding=True)


with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Reference:", test_dataset[i]["sentence"])
    print("Prediction:", predicted_sentence)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-polish were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-polish and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You

TypeError: Invalid file: 0    C:\Users\Eryk\.cache\huggingface\datasets\down...
1    C:\Users\Eryk\.cache\huggingface\datasets\down...
2    C:\Users\Eryk\.cache\huggingface\datasets\down...
3    C:\Users\Eryk\.cache\huggingface\datasets\down...
4    C:\Users\Eryk\.cache\huggingface\datasets\down...
Name: audiopath_local, dtype: object